In [1]:
import pandas as pd
import numpy as np

red = pd.read_csv('winequality-red.csv', sep=';')
white = pd.read_csv('winequality-white.csv', sep=';')

FileNotFoundError: [Errno 2] File b'winequality-red.csv' does not exist: b'winequality-red.csv'

In [ ]:
#red.head(2)
print(red.shape)

In [ ]:
#white.head(2)
print(white.shape)

In [ ]:
missing_values_red = red.isnull().sum().sort_values(ascending=False)
missing_values_red.head()

In [ ]:
missing_values_white = white.isnull().sum().sort_values(ascending=False)
missing_values_white.head()

In [ ]:
review_red = pd.read_csv('winemag-data_first150k.csv')

In [ ]:
review_red = review_red.drop(['country','designation','points',\
                             'price','province','region_1','region_2',\
                             'variety','winery'], axis=1)
review_red.rename(columns={'Unnamed: 0':'Number'}, inplace= True)

In [ ]:
#review_red.head(2)
print(len(review_red))
red_reviews = pd.Series(review_red.description.values, index = review_red.Number).to_dict()

In [ ]:
red['review'] = np.random.randint(0,len(review_red),red.shape[0])

In [ ]:
red['review'] = red['review'].map(red_reviews)

In [ ]:
red.head(2)

In [ ]:
review_white = pd.read_csv('winemag-data-130k-v2.csv')

In [ ]:
review_white = review_white.drop(['country','designation','points',\
                             'price','province','region_1','region_2',\
                             'variety','winery','taster_name',\
                                 'taster_twitter_handle','title'], axis=1)
review_white.rename(columns={'Unnamed: 0':'Number'}, inplace= True)

In [ ]:
print(len(review_white))
white_reviews = pd.Series(review_white.description.values, index = review_white.Number).to_dict()

In [ ]:
white['review'] = np.random.randint(0,len(review_white),white.shape[0])

In [ ]:
white['review'] = white['review'].map(white_reviews)

In [ ]:
white.head(2)

In [ ]:
red.to_csv("red_wine_with_review.csv")
white.to_csv("white_wine_with_review.csv")

## Red Wine

In [ ]:
# fetch wordcount for each review
red['review_word_count'] = red['review'].apply(lambda x: len(str(x).split(" ")))
red[['review','review_word_count']].head(2)

In [ ]:
red.review_word_count.describe()

In [ ]:
# 10 most common words
frequency = pd.Series(' '.join(red['review']).split()).value_counts()[:10]
#frequency
# 10 most uncommon words
frequency_un = pd.Series(' '.join(red['review']).split()).value_counts()[-10:]
#frequency_un

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
import nltk
from nltk import corpus
nltk.download("stopwords")
nltk.download('wordnet') 


In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop_words = stopwords.words('english')

In [ ]:
corpus = []
for i in range(0, len(red)):
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', red['review'][i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)

In [ ]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt


wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stop_words,
                          max_words=100,
                          max_font_size=50, 
                          random_state=42
                         ).generate(str(corpus))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=900)

In [ ]:
#cv = CountVectorizer(max_df=0.85, stop_words = stop_words, max_features=10000)
#word_count_vector = cv.fit_transform(list(red['review']))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    min_df=5, max_features=100, strip_accents='unicode',lowercase =True,
    analyzer='word', token_pattern=r'\w+', use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words = 'english').fit(red['review'])

In [ ]:
features = tfidf.get_feature_names()
print(features)

## White Wine

In [ ]:
white['review_word_count'] = white['review'].apply(lambda x: len(str(x).split(" ")))
white[['review','review_word_count']].head(2)

In [ ]:
white.review_word_count.describe()

In [ ]:
corpus = []
for i in range(0, len(white)):
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', white['review'][i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)

In [ ]:
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stop_words,
                          max_words=100,
                          max_font_size=50, 
                          random_state=42
                         ).generate(str(corpus))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("white_keywords.png", dpi=900)

In [ ]:
tfidf = TfidfVectorizer(
    min_df=5, max_features=100, strip_accents='unicode',lowercase =True,
    analyzer='word', token_pattern=r'\w+', use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words = 'english').fit(white['review'])

In [ ]:
features1 = tfidf.get_feature_names()
print(features1)